# Optimistic initial values (A k-armed bandit problem)
by __Shivangi Agarwal and Sandeep Banik__ | Reinforcement Learning

In the k-armed bandit problem, it was shown how to estimate $Q(a)$ values for each bandit and the corresponding lever/action $(a)$. However, it is observed that the estimates are biased on the initial values. This is explained below by expanding the update of each estimate.

<img src="OIV-inital bias.png">

Although the use of sample average ($\alpha = \frac{1}{n}$) helps in rectifying the bias issue, however for a general constant step-size $\alpha$ the initial values need to be dealt with. Here we show that picking an inital value can lead to exploration.  

To simply our life let us assume that for the k-armed bandit, we see that the $q_{*}$ were selected randomly from a uniform distribution with a mean of 0 and variance 1. That means that the expected value of the actual reward we achieved by selecting a lever will be between -1 and 1 and a variance of 1 about that mean. Let us say for a 5 armed bandit, the mean of the rewards are 0.1, 0.2, 0.3, 0.4 and 0.5 respectively.  

Now optimistic inital value states that, we assume some really 'optimistic value' let's say 5 for each of the 5 arms. If we select such a value we can observe that this is much higher than what value we know about the arms. Let us begin playing the slot machine being greedy about our $Q$ values. 

<img src="OIV-5armed-bandit.png">

You can see that the first time it played it chose the action 1 and it got disappointed and picked another lever to expect a better reward. The optimistic inital approach naturally choses to explore all possible actions. The technique is called encouraging exploration optimistic initial value. However this technique is not suited for non-stationary problems because the exploration as observed is present only in the beginning. In a non-stionary problem the mean of the reward might change over time demanding exploration at a later on stage.